## Scenario 3: Tracking server on localhost with SQLite backend and remote (s3) artifacts store

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root s3://<YOUR_BUCKET>`

The exampe uses an s3 bucket to store the artifacts, so you'll to provide the bucket. Check the [mlflow documentation](https://mlflow.org/docs/latest/tracking.html#artifact-stores) for more options to store your artifacs.

In [1]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "pana" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_HOST_SERVER = 'ec2-XX-XXX-XX-XXX.compute-1.amazonaws.com'
mlflow.set_tracking_uri(f"http://{TRACKING_HOST_SERVER}:5000")

In [3]:
print(f"tracking uri: {mlflow.get_tracking_uri()}")

tracking uri: http://ec2-XX-XXX-XX-XXX.compute-1.amazonaws.com:5000


In [5]:
mlflow.list_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-store/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-store/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>]

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")

/home/ubuntu/anaconda3/envs/exp-tracking-env-week2/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [7]:
mlflow.list_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-store/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-store/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>]

### Interacting with the model registry

In [8]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_HOST_SERVER}:5000")

In [9]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1654680828749, description='', last_updated_timestamp=1654680828780, latest_versions=[<ModelVersion: creation_timestamp=1654680828780, current_stage='None', description='', last_updated_timestamp=1654680828780, name='iris-classifier', run_id='7d780f443d2f4beb9753d3c5f9cce5f9', run_link='', source='s3://mlflow-artifacts-remote-store/1/7d780f443d2f4beb9753d3c5f9cce5f9/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>]

In [10]:
run_id = client.list_run_infos(experiment_id='1')[0].run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier-1'
)

Successfully registered model 'iris-classifier-1'.
2022/06/08 10:59:24 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris-classifier-1, version 1
Created version '1' of model 'iris-classifier-1'.


<ModelVersion: creation_timestamp=1654685964093, current_stage='None', description='', last_updated_timestamp=1654685964093, name='iris-classifier-1', run_id='332a813ecebb40bb81d9b2da9dcb2651', run_link='', source='s3://mlflow-artifacts-remote-store/1/332a813ecebb40bb81d9b2da9dcb2651/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

In [11]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1654680828749, description='', last_updated_timestamp=1654680828780, latest_versions=[<ModelVersion: creation_timestamp=1654680828780, current_stage='None', description='', last_updated_timestamp=1654680828780, name='iris-classifier', run_id='7d780f443d2f4beb9753d3c5f9cce5f9', run_link='', source='s3://mlflow-artifacts-remote-store/1/7d780f443d2f4beb9753d3c5f9cce5f9/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>,
 <RegisteredModel: creation_timestamp=1654685964064, description='', last_updated_timestamp=1654685964093, latest_versions=[<ModelVersion: creation_timestamp=1654685964093, current_stage='None', description='', last_updated_timestamp=1654685964093, name='iris-classifier-1', run_id='332a813ecebb40bb81d9b2da9dcb2651', run_link='', source='s3://mlflow-artifacts-remote-store/1/332a813ecebb40bb81d9b2da9dcb2651/artifacts/models', status='READY', status_message='', tags={